In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time 

from ipynb.fs.full.Simulator_with_var_reduction import Variance_Reduction_simulate



def neighbors(u):
    space = []

    for i in range(4):
        for du in [-1,1]:
            dus=u*1;
            dus[i]=u[i]+du
            if dus[i]>=0 and dus[3]>=1 and 2*dus[0]+3*dus[1]+4*dus[2]+5*dus[3]<=400:
                space.append(dus)
    return space

# function for claculating the cost/objective function, 
#queue_weight=0 for Optimizing revenue only, = 1000 for optimizing both revenue and queue length
def cost_function(max_queue_mean,Rev_mean,queue_weight=0):
    return -1*Rev_mean+max_queue_mean*queue_weight


def temperature(max_reduct,reduct_times,p_max,p_final,typ_increase):
    reduct_times=min([max_reduct,reduct_times])
    nom = np.log(p_max+(p_final-p_max)*reduct_times/max_reduct)
    return -typ_increase/nom

def Optimize(start_point,max_iteration,max_reduct,p_max,p_final,typ_increase,run_times):
    iter=0;
    u=start_point;

    max_queue_mean,Rev_mean = Variance_Reduction_simulate(u,run_times)
    fu=cost_function(max_queue_mean,Rev_mean)
    
    Opt_u=u;
    Opt_f=fu;
    Opt_Rev=Rev_mean;
    Opt_max_que=max_queue_mean;
    
    log_u=[];
    log_T=[];
    log_f=[];
    
    log_Rev=[];
    log_max_que=[];
    
    log_Opt_f=[];
    log_Opt_u=[];
    log_Opt_Rev=[];
    log_Opt_max_que=[];
    t0=time.time();
    
    
    while iter<=max_iteration:
        clear_output(wait=True)
        T=temperature(max_reduct,iter,p_max,p_final,typ_increase)
        
        fu_=[]
        all_max_queue_mean_=[]
        all_Rev_mean_=[]
        
        Vu=neighbors(u);
        for u_ in Vu:   
            
            max_queue_mean_,Rev_mean_=Variance_Reduction_simulate(u_,run_times)
            fu_.append(cost_function(max_queue_mean_,Rev_mean_))
            all_max_queue_mean_.append(max_queue_mean_)
            all_Rev_mean_.append(Rev_mean_)
            
        fu_min=min(fu_);
        
        if fu_min<=fu or np.exp((fu-fu_min)/T)>np.random.rand():
            fu = fu_min;
            u = Vu[fu_.index(fu_min)]
            max_queue_mean=all_max_queue_mean_[fu_.index(fu_min)]
            Rev_mean=all_Rev_mean_[fu_.index(fu_min)]
            
        if fu<Opt_f:
            Opt_u=u;
            Opt_f=fu;
            Opt_Rev=Rev_mean;
            Opt_max_que=max_queue_mean;
        
        log_u.append(u)
        log_T.append(T)
        log_f.append(fu)
        
        log_Rev.append(Rev_mean);
        log_max_que.append(max_queue_mean);
        
        log_Opt_f.append(Opt_f)
        log_Opt_u.append(Opt_u)
        log_Opt_Rev.append(Opt_Rev);
        log_Opt_max_que.append(Opt_max_que);
        
        
        print('Optimization Process:',end="")
        print("{:.2%}".format(iter*1.00/(max_iteration*1.00)))
        print('Time spent:',end="")
        print("{:.2f}".format(time.time()-t0))

#         time.sleep(0.01)
       
        iter+=1;        
       
    return log_u,log_T,log_f,log_Rev,log_max_que,log_Opt_f,log_Opt_u,log_Opt_Rev,log_Opt_max_que,time.time()-t0

# print('Averaged Final revenue')
# print(average_simulate(u_0,100))

In [ ]:
u_0 = [19,18,4,20] #Start point
log_u,log_T,log_f,log_Rev,log_max_que,log_Opt_f,log_Opt_u,log_Opt_Rev,log_Opt_max_que,time=Optimize(u_0,1000,800,0.99,0.01,300,124)

Optimization Process:65.90%
Time spent:4760.37


In [ ]:
#Visualization

iters=range(len(log_u));

fig,ax=plt.subplots()
ax.plot(iters, log_Rev, color="orange", linewidth=1,label="Iteration value")
ax.plot(iters, log_Opt_Rev,label="Optimal value")
ax.set_xlabel("iterations",fontsize=14)
ax.set_ylabel("Estimated Revenue",fontsize=14)
ax.legend(bbox_to_anchor =(0.7, 1.15), ncol = 2)

ax2=ax.twinx()
ax2.plot(iters, log_T,color="green",label="Temperature")
ax2.set_ylabel("Temperature",fontsize=14)
ax2.legend(bbox_to_anchor =(1, 1.15), ncol = 1)

ax.grid()

fig,ax0=plt.subplots()
ax0.plot(iters, log_max_que, color="pink", linewidth=1,label="Iteration value")
ax0.plot(iters, log_Opt_max_que,color="black",label="Optimal value")
ax0.set_xlabel("iterations",fontsize=14)
ax0.set_ylabel("Estimated Max Queue Length",fontsize=14)
ax0.legend(bbox_to_anchor =(0.7, 1.15), ncol = 2)

ax1=ax0.twinx()
ax1.plot(iters, log_T,color="red",label="Temperature")
ax1.set_ylabel("Temperature",fontsize=14)
ax1.legend(bbox_to_anchor =(1, 1.15), ncol = 1)


ax0.grid()
plt.show()
print("Optimized Table arrangement:",end="")
print(log_Opt_u[-1])
print("Corresponding Revenue:",end="")
print("{:.2f}".format(log_Opt_Rev[-1]),end="")
print("CHF")
print("Corresponding max queue length:",end="")
print(log_Opt_max_que[-1])
print("Running time:",end="")
print("{:.2f}".format(time),end="")
print(" seconds")
